# Linear Regression on Review Keywords and Ratings
In this part we try to determin ratings based on key words from reviews

      Questions: 
    
    Are key words good indications of a ratings?
    Can we build a model to predict a rating based on a group of key words?
Son Phan 862042081
Edward Dickoff 861167330
 

In [1]:
import pandas as pd
import re
import nltk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.pyplot import figure

df_orig = pd.read_csv('seaworld_complete.csv')
df = df_orig
df
df['Review'].fillna('', inplace=True) # fill Nulls with empty space

In [2]:
df.replace({'False': 0, 'Ture': 1}, inplace=True)
df = df[:2479]
df.tail()

#First thing needed is to change true/false to integers for analysis
df.replace({False: 0, True: 1}, inplace=True)
df.head()
df.tail()

/usr/local/lib64/python3.6/site-packages/pandas/core/frame.py:4278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,Number,Date,Rating,Review,great,happy,fantastic,lovely,good,enjoyed,...,whale,whales,porpoise,captivity,cages,cage,blackfish,shamu,group2,year_bin
2474,9,2020-02-28,5.0,this place was so fun! great way to spend my 3...,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2020.0
2475,30,2020-02-28,1.0,"i love seaworld however, their web site is the...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2020.0
2476,0,2020-02-28,4.0,**bags and backpacks are not allowed on some r...,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,2020.0
2477,45,2020-02-29,1.0,gregory and rafael were no help at customer se...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2020.0
2478,7,2020-03-01,5.0,"when you come to sea world, look for ben! he's...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2020.0


In [3]:
#drop unneeded columns
df = df.drop(columns = ['Review', 'Date', 'Number', 'year_bin'])
df.head()


,Rating,great,happy,fantastic,lovely,good,enjoyed,loved,amazing,informative,...,mammals,whale,whales,porpoise,captivity,cages,cage,blackfish,shamu,group2
0,4.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4.0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,1
2,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
#seperate data into test data
df_train = df[:2400]
df_test = df[2400:]
df_test.head()

,Rating,great,happy,fantastic,lovely,good,enjoyed,loved,amazing,informative,...,mammals,whale,whales,porpoise,captivity,cages,cage,blackfish,shamu,group2
2400,2.0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2401,4.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2402,4.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2403,4.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2404,3.0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
df = pd.read_csv('seaworlds.csv')
df['Review'] = df['Review'].str.lower() 
df.head()

,Number,Date,Rating,Review
0,0,2/28/2020,4.0,**bags and backpacks are not allowed on some r...
1,1,2/21/2020,4.0,family went on a weekday which was wonderful a...
2,2,2/1/2020,5.0,we always have a great time. â we have season ...
3,3,1/3/2020,5.0,considering i live in san diego this is pretty...
4,4,12/7/2019,4.0,"sea world is adding more rides, attractions â ..."


In [6]:
df = pd.read_csv('seaworlds.csv')
df['Review'] = df['Review'].str.lower() 
df.tail()

,Number,Date,Rating,Review
2476,2475,9/25/2011,4.0,really hard to enjoy in the summer because its...
2477,2476,3/9/2010,3.0,i went to sea world on a sunday afternoon. the...
2478,2477,3/14/2011,2.0,i went sea world recently - probably my first ...
2479,2478,8/2/2011,5.0,i thought seaworld was a fantastic amusement p...
2480,2479,2/18/2011,5.0,i love sea world. â the shows are great... i l...


In [7]:
df_train = df[:2300]
#df_train=df_train          
df_test = df[2300:]
#df_test=df_test
df_train.shape, df_test.shape

((2300, 4), (181, 4))

In [8]:
#df_train = df_train[['Review', 'Rating']]
#df_train['Rating'].hist();df_train.head()

In [9]:

max_features = 100
tfidf = TfidfVectorizer(max_features = max_features)

NameError: name 'TfidfVectorizer' is not defined

In [ ]:
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(df_train["Review"])

df_train_tfidf = pd.DataFrame(tfidf_vectorizer_vectors.todense(),columns=tfidf_vectorizer.get_feature_names())
df_train_tfidf.head()

In [ ]:
df_train = pd.concat([df_train, df_train_tfidf], axis=1)
df_train.head()

In [ ]:
X_train = df_train.copy(deep=True) # copy the dataframe
X_train = X_train[["whale","went"]] # use only 1 feature for trainings to start off 

X_train.head(2)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression


tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(df_test["Review"])

df_test_tfidf = pd.DataFrame(tfidf_vectorizer_vectors.todense(),columns=tfidf_vectorizer.get_feature_names())
df_test_tfidf.head()

df_test = pd.concat([df_test, df_test_tfidf], axis=1)
df_test.head()



X_test = df_test 
print(X_test.animals)
X_test = X_test[["whale","went"]] # use only 1 feature for trainings to start off 


y_train = df_train.Rating

model = LinearRegression()
model.fit(X=X_train, y=y_train)
Y_predict = model.predict(X=X_test)
Y_predict